# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
# !pip install cartopy
# !pip install geoviews
# !pip install pyproj
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,19.87,84,93,9.39,PT,1698714251
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,5.33,62,0,6.00,RU,1698714251
2,2,kodiak,57.7900,-152.4072,7.73,93,100,3.60,US,1698714251
3,3,whitehorse,60.7161,-135.0538,1.72,93,100,3.60,CA,1698714251
4,4,oleksandriya,48.6696,33.1159,12.37,81,11,4.73,UA,1698714252


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot

city_data_df = city_data_df.dropna()
city_data_df["Lat"] = city_data_df["Lat"].astype(float)
city_data_df["Lng"] = city_data_df["Lng"].astype(float)
city_data_df["Humidity"] = city_data_df["Humidity"].astype(float)



city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    size = "Humidity",
    color = "City",
    hover_cols = ["City", "Country"]
)

# Display the map
city_humidity_map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df

#Cities with weather between 21 and 27 degrees celsius
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] > 21) & (ideal_weather_df["Max Temp"] < 27)]

#Cities with wind less than 4.5m/s
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 4.5]

#Cities with zero cloudiness
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,lompoc,34.6391,-120.4579,21.08,59.0,0,4.02,US,1698714270
115,115,arraial do cabo,-22.9661,-42.0278,24.92,88.0,0,1.54,BR,1698714158
126,126,ghat,24.9647,10.1728,23.05,19.0,0,1.01,LY,1698714158
138,138,fort bragg,35.1390,-79.0060,23.79,65.0,0,3.60,US,1698714226
234,234,ormara,25.2088,64.6357,26.25,64.0,0,1.55,PK,1698714305
237,237,pacific grove,36.6177,-121.9166,21.06,51.0,0,1.79,US,1698714010
247,247,salalah,17.0151,54.0924,25.05,83.0,0,2.06,OM,1698714309
305,305,puerto penasco,31.3167,-113.5333,26.06,14.0,0,2.24,MX,1698714329
371,371,shenjiamen,29.9576,122.2980,21.71,62.0,0,2.55,CN,1698714351
394,394,formosa do rio preto,-11.0483,-45.1931,26.29,33.0,0,1.22,BR,1698714355


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df.drop(columns=["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis=1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
68,lompoc,34.6391,-120.4579,59.0,US,
115,arraial do cabo,-22.9661,-42.0278,88.0,BR,
126,ghat,24.9647,10.1728,19.0,LY,
138,fort bragg,35.1390,-79.0060,65.0,US,
234,ormara,25.2088,64.6357,64.0,PK,
237,pacific grove,36.6177,-121.9166,51.0,US,
247,salalah,17.0151,54.0924,83.0,OM,
305,puerto penasco,31.3167,-113.5333,14.0,MX,
371,shenjiamen,29.9576,122.2980,62.0,CN,
394,formosa do rio preto,-11.0483,-45.1931,33.0,BR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "types": "Hotel",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {lat}, {lng}, {radius}"
    params["bias"] = f"proximity:{lat}, {lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(name_address)
hotel_df

Starting hotel search
lompoc - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
pacific grove - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
puerto penasco - nearest hotel: No hotel found
shenjiamen - nearest hotel: No hotel found
formosa do rio preto - nearest hotel: No hotel found
bangui - nearest hotel: No hotel found
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}


,City,Lat,Lng,Humidity,Country,Hotel Name
68,lompoc,34.6391,-120.4579,59.0,US,No hotel found
115,arraial do cabo,-22.9661,-42.0278,88.0,BR,No hotel found
126,ghat,24.9647,10.1728,19.0,LY,No hotel found
138,fort bragg,35.1390,-79.0060,65.0,US,No hotel found
234,ormara,25.2088,64.6357,64.0,PK,No hotel found
237,pacific grove,36.6177,-121.9166,51.0,US,No hotel found
247,salalah,17.0151,54.0924,83.0,OM,No hotel found
305,puerto penasco,31.3167,-113.5333,14.0,MX,No hotel found
371,shenjiamen,29.9576,122.2980,62.0,CN,No hotel found
394,formosa do rio preto,-11.0483,-45.1931,33.0,BR,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot

hotel_df["Lat"] = hotel_df["Lat"].astype(float)
hotel_df["Lng"] = hotel_df["Lng"].astype(float)




hotel_map_df = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    size = "Humidity",
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)

# Display the map
hotel_map_df

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)